In [45]:
# Assuming there are no transactions taking place unless an until place_market_order function is called
class Limit:
    
    i=0      # global variable for unique order id assignment
    ask=[]   # limit order list of sellers
    bid=[]   # limit order list of buyers
    
    pq_ask = [] # list of quantity-price pairs in order of best offer
    pq_bid = [] # list of quantity-price pairs in order of best bid
    
    
    def func1(self,x):
        return (x[1])
    
    def func2(self,x):
        return (x[1])
    
    
    def add_limit_order(self,side,user_id,quantity,price):
        
        self.i+=1
        
        d={ "order_id" : self.i,
            "user_id" : user_id,
            "quantity" : quantity,
            "price" : price }
        
        if(side is 'ask'):
            self.ask.append(d)
            self.pq_ask.append([d["quantity"], d["price"]])
        
        
        if(side is 'bid'):
            self.bid.append(d);
            self.pq_bid.append([d["quantity"],d["price"]])
        
        
        self.pq_ask = sorted(self.pq_ask,key=self.func1,reverse=False)
    
        self.pq_bid = sorted(self.pq_bid,key=self.func2,reverse=True)
        
    
    def showBook(self):
    
        print(self.ask)
        
        print(self.bid)
        
    
    def place_market_order(self,side,quantity):
        
        currentNumberOfSellers = len(self.pq_ask)
        currentNumberOfBuyers = len(self.pq_bid)
        
        if(side is 'ask'):
            
            totalPrice = 0
            initialQuantity = quantity
            
            for i in range (currentNumberOfSellers):
                
                x = self.pq_ask[i]
                
                if(x[0] >= quantity):
                    
                    totalPrice += (x[1]*quantity)
                    self.pq_ask[i][0] -= quantity
                    quantity = 0
                    break
                
                if(x[0] < quantity):
                    
                    totalPrice += (x[1]*x[0])
                    quantity -= x[0]
                    self.pq_ask[i][0] = 0
                
                if(quantity == 0):
                    break
                    
            quantitySold = initialQuantity - quantity
            avgSellingPrice = totalPrice / quantitySold
            
            return quantitySold, avgSellingPrice
        
        if(side is 'bid'):
            
            totalPrice = 0
            initialQuantity = quantity
            
            for i in range (currentNumberOfBuyers):
                
                x=self.pq_bid[i]
                
                if(x[0] >= quantity):
                    
                    totalPrice += (x[1]*quantity)
                    self.pq_bid[i][0] -= quantity
                    quantity=0
                    break
                    
                if(x[0] < quantity):
                    
                    totalPrice += (x[1]*x[0])
                    quantity -= x[0]
                    self.pq_bid[i][0] = 0
                    
                if (quantity==0):
                    break
                
            quantityBought = initialQuantity - quantity
            avgBuyingPrice = totalPrice / quantityBought
            
            #arranging list again in order of best bid/offer after updation
            self.pq_ask = sorted(self.pq_ask,key=self.func1,reverse=False)
            self.pq_bid = sorted(self.pq_bid,key=self.func2,reverse=True)
                
            return quantityBought,avgBuyingPrice
        
    def cancel_limit_order(self,order_id):
        
        numberOfSellers = len(self.pq_ask)
        numberOfBuyers = len(self.pq_bid)
        flag = 0
        
        for i in range (numberOfSellers):
            
            x = self.ask[i]
            
            if(x["order_id"] == order_id):
                flag = 1
                break
        
        if(flag == 0):
            for i in range (numberOfBuyers):
                
                x = self.bid[i]
            
                if(x["order_id"] == order_id):
                    
                    flag = 1
                    break
        
            del self.bid[i]
            
        else:
            
            del self.ask[i]
    
    def bbo(self):
        
        x = self.pq_ask[0]
        y = self.pq_bid[0]
        
        return y[1],x[1]

In [46]:
lob=Limit()

In [47]:
lob.add_limit_order('ask', 'alice', 10, 100)
lob.add_limit_order('ask', 'bob', 5, 95)
lob.add_limit_order('bid', 'charles', 20, 85)
lob.add_limit_order('bid', 'dave', 10, 80)
#lob.add_limit_order('ask', 'harry',12, 75)
#print(lob.pq_ask)
#lob.add_limit_order()


In [48]:
lob.showBook()

[{'order_id': 1, 'user_id': 'alice', 'quantity': 10, 'price': 100}, {'order_id': 2, 'user_id': 'bob', 'quantity': 5, 'price': 95}]
[{'order_id': 3, 'user_id': 'charles', 'quantity': 20, 'price': 85}, {'order_id': 4, 'user_id': 'dave', 'quantity': 10, 'price': 80}]


In [49]:
lob.place_market_order('ask',12)

(12, 97.91666666666667)

In [50]:
lob.cancel_limit_order(4)

In [51]:
lob.showBook()


[{'order_id': 1, 'user_id': 'alice', 'quantity': 10, 'price': 100}, {'order_id': 2, 'user_id': 'bob', 'quantity': 5, 'price': 95}]
[{'order_id': 3, 'user_id': 'charles', 'quantity': 20, 'price': 85}]


In [52]:
lob.bbo()

(85, 95)